# Finance with Python

## Data Gathering and Preparation

This notebook is the first part of our Finance with Python study. 

It contains an automation of the data gathering process by :
- Scrapping the tickers symbols of the **S&P 500 index** companies.
- Gathering the Stock prices of these companies using the **Yahoo! Finance API**.
- Merging the Adjacent Closing Price of each companie into one dataset.

## 1 Defintions

#### S&P 500
The S&P 500, or simply the S&P, is a stock market index that measures the stock performance of 500 large companies listed on stock exchanges in the United States. It is one of the most commonly followed equity indices, and many consider it to be one of the best representations of the U.S. stock market.

Source : https://en.wikipedia.org/wiki/S%26P_500_Index

#### Ticker Symbol
A ticker symbol or stock symbol is an abbreviation used to uniquely identify publicly traded shares of a particular stock on a particular stock market. A stock symbol may consist of letters, numbers or a combination of both. 

Exemple: **AAPL** for Apple Inc, **NFLX** for Netflix Inc.

Source : https://en.wikipedia.org/wiki/Ticker_symbol


#### Adjusted Closing Price
Adjusted closing price amends a stock's closing price to accurately reflect that stock's value after accounting for any corporate actions. It is considered to be the true price of that stock and is often used when examining historical returns or performing a detailed analysis of historical returns.

Source : https://www.investopedia.com/terms/a/adjusted_closing_price.asp

## 2 Resources

- List of the S&P 500 companies : https://en.wikipedia.org/wiki/List_of_S%26P_500_companies
- How to download stock prices in Python : https://www.codingfinance.com/post/2018-03-27-download-price-py/
- How to calculate stock returns in Python : https://www.codingfinance.com/post/2018-04-03-calc-returns-py/
- Sentdex YouTube Playlist, Python Programming for Finance : https://www.youtube.com/playlist?list=PLQVvvaa0QuDcOdF96TBtRtuQksErCEBYZ
- Computer Science YouTube video Python for Finance : https://www.youtube.com/watch?v=O-O1WclwXck
- Financial Python : https://www.quantconnect.com/tutorials/introduction-to-financial-python/rate-of-return,-mean-and-variance
- Yahoo Finance API to get Stocks tickers data in python : https://medium.com/@jouneidraza522/yahoo-finance-api-to-get-stocks-tickers-data-in-python-c49820249a18

## 3 Loading Modules

In [1]:
import numpy as np  # Scientific computing
import pandas as pd # Data analysis and manipulation


from datetime import datetime as dt  # Dates and times manipulations

import pandas_datareader.data as web # Up to date remote data access for pandas 
                                     # pip install pandas-datareader

import bs4 as bs # Pulling data out of HTML and XML files
import pickle    # Serializing and de-serializing a Python object structure.
import requests  # Send HTTP requests easily
import os        # operating system dependent functionality.

import yfinance as yf # Historical market data from Yahoo! finance

## 4 Scrapping the Tickers symbols of the S&P 500 index companies


We will : 
- Send an HTTP Request to the wikipedia page **List of the S&P 500 companies** : https://en.wikipedia.org/wiki/List_of_S%26P_500_companies .
- Scrap the tickers data out of the HTML using BeautifulSoup4.
- serialize a Python object using Pickle dump function.

In [2]:
# Creating a function to get the 500 tickers of the S&P compagnies
def get_sp500_tickers():
    
    # Sending a requests to the wikipedia page and storing the response
    resp = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    
    # Creating the BeautifulSoup object, using the text of the response and lxml as a parser 
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    
    # Find the table data by class or id (Preferable)
    #table = soup.find('table', {'class':'wikitable sortable'})
    table = soup.find('table', {'id': 'constituents'})
    
    # Creating an empty list to store the tickers
    tickers = []
    
    # Looping through all the table rows, except the first table row, it contains the title of the columns
    for row in table.findAll('tr')[1:]:
        
        # Assign the first column (table data)[0] of each row to a ticker variable
        # Some of the stock tickers output a period '.' instead of a hyphen '-' (Yahoo! Finance API)
        ticker = row.findAll('td')[0].text.replace('.', '-')
        
        # Getting rid of the \n
        ticker = ticker[:-1]
    
        # Append the ticker to the tickers list
        tickers.append(ticker)
    
    # Creating the pickle file, with (write binary) mode
    with open('sp500_tickers.pickle', 'wb') as file:
        
        # Dumping the tickers into the pickle file 
        pickle.dump(tickers, file)
    
    return tickers

In [3]:
# Getting the first 10 tickers symbols 
get_sp500_tickers()[:10]

['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP', 'AES']

## 5 Gathering the Stock prices using the Yahoo! Finance API

We will :
- Create a "data" directory if it doesn't exist.
- De-serialize the Pickle file using the load function in order to get the latest tickers in the S&P 500 list.
- Gather the data of each companie from the Yahoo! Finance API using the get_data_yahoo() function.
- Store the gathered data in a ticket.csv file.

#### Gathering the 500 companies stock prices requires a lot of computational time, you might need to restart this cell.

In [4]:
# Creating a function to get the sp500 data from yahoo and stores it locally 
def get_sp500_data():
  
    # Opening the sp500_tickers.pickle file with (read binary) mode
    with open('sp500_tickers.pickle', 'rb') as file:
            
        # Loading the tickers from the file
        tickers = pickle.load(file)
    
    # Cheking if the data directory exists, else creates it
    if not os.path.exists('data'):
        os.makedirs('data')
        
    # Setting up the starting and ending date
    start = dt(2010, 1, 1)
    end = dt.now() # 2020-03-27
    
    # Looping through the symbols
    # Add [:int < 500] to reduce the amount of data and time 
    for ticker in tickers:
        
        # Cheking if the ticker.csv file does not exist 
        if not os.path.exists('data/{}.csv'.format(ticker)):
            
            # Get the data from Yahoo
            df = web.get_data_yahoo(ticker, start, end)
             
            # We reset the actual index
            df.reset_index(inplace=True)
            
            # We set in Date as the index
            df.set_index("Date", inplace=True)
            
            # Saving the ticker.csv file
            df.to_csv('data/{}.csv'.format(ticker))
        # If the symbol.csv file exists
        else:
            print('Already have {}'.format(ticker))

In [5]:
# Gathering the sp500 data
get_sp500_data()

Already have MMM
Already have ABT
Already have ABBV
Already have ABMD
Already have ACN
Already have ATVI
Already have ADBE
Already have AMD
Already have AAP
Already have AES
Already have AFL
Already have A
Already have APD
Already have AKAM
Already have ALK
Already have ALB
Already have ARE
Already have ALXN
Already have ALGN
Already have ALLE
Already have AGN
Already have ADS
Already have LNT
Already have ALL
Already have GOOGL
Already have GOOG
Already have MO
Already have AMZN
Already have AMCR
Already have AEE
Already have AAL
Already have AEP
Already have AXP
Already have AIG
Already have T
Already have AMT
Already have AWK
Already have AMP
Already have ABC
Already have AME
Already have AMGN
Already have APH
Already have ADI
Already have ANSS
Already have ANTM
Already have AON
Already have AOS
Already have APA
Already have AIV
Already have AAPL
Already have AMAT
Already have APTV
Already have ADM
Already have ARNC
Already have ANET
Already have AJG
Already have AIZ
Already have AT

## 6 Merging all the S&P 500 data into one DataFrame 

We will:
- De-serialize the Pickle file in order to get the latest tickers in the S&P 500 list, not all the files in the data directory.
- Read every dataset, store the Adjacent Closing Price and merge it with the previous one.
- Create one S&P 500 Adjacent Closing Price Dataset and save it locally.

In [8]:
# Creating a funtion that will merge all the datasets into one
def merge_data():
    
    # Opening the sp500_tickers.pickle file with (read binary) mode
    with open('sp500_tickers.pickle', 'rb') as file:
            
        # Loading the tickers from the file
        tickers = pickle.load(file)
    
    # Creating an empty dataframe to store all the data    
    df = pd.DataFrame()
    
    # Looping through the tickers
    for ticker in tickers:
        
        # Reading the symbol.csv file
        df_ticker = pd.read_csv('data/{}.csv'.format(ticker))
        
        # Setting the index to Date
        df_ticker.set_index('Date', inplace = True)
        
        # Renaming the Adj Close to be the symbol
        df_ticker.rename(columns = {'Adj Close' : ticker}, inplace = True)
        
        #Dropping the unwanted columns
        df_ticker.drop(['Open', 'High', 'Low', 'Close', 'Volume'], axis = 1, inplace = True)
        
        # Joining the dataframe with the main_df
        df = df.join(df_ticker, how = 'outer', sort = False)
      
    print(df.head())
        
    # Creating the sp500_joined_closes.csv file
    df.to_csv('sp500_adj_close.csv') 

In [9]:
# Merging the data
merge_data()

                  MMM        ABT  ABBV  ABMD        ACN       ATVI       ADBE  \
Date                                                                            
2010-01-04  63.519070  18.229385   NaN  8.74  33.871223  10.193225  37.090000   
2010-01-05  63.121239  18.082108   NaN  8.53  34.080551  10.211267  37.700001   
2010-01-06  64.016403  18.182526   NaN  8.40  34.442860  10.157144  37.619999   
2010-01-07  64.062294  18.333153   NaN  8.40  34.410656   9.913588  36.889999   
2010-01-08  64.513718  18.426876   NaN  8.23  34.273785   9.832404  36.689999   

             AMD        AAP        AES  ...       WYNN        XEL        XRX  \
Date                                    ...                                    
2010-01-04  9.70  39.293575  10.778586  ...  41.963718  14.407637  19.292725   
2010-01-05  9.71  39.060036  10.668198  ...  44.515926  14.236770  19.315083   
2010-01-06  9.57  39.400608  10.557810  ...  43.932011  14.264105  19.136232   
2010-01-07  9.47  39.390865  10.

#### The missing values (NaN) mean that the companies were not listed on the stock exchange or were not part of the S&P 500 Index.